In [1]:
from osgeo import gdal, ogr
import json
import subprocess
import os

In [2]:
def getCenter(arr):
    # array format: [xmin, ymin, xmax, ymax]
    xmin = arr[0]
    ymin = arr[1]
    xmax = arr[2]
    ymax = arr[3]
    x = (xmin + xmax) / 2
    y = (ymin + ymax) / 2
    return x, y

In [3]:
def getBoundingBox (INPUT_IMAGE):
    src = gdal.Open(INPUT_IMAGE)
    ulx, xres, xskew, uly, yskew, yres = src.GetGeoTransform()
    xmin = ulx
    ymin = uly + (src.RasterYSize * yres)
    xmax = ulx + (src.RasterXSize * xres)
    ymax = uly
    
    res = []
    res.append(xmin)
    res.append(ymin)
    res.append(xmax)
    res.append(ymax)
    return res

In [4]:
# returns True if INSIDE is inside OUTSIDE
def checkBounds (INSIDE, OUTSIDE):
    # INSIDE, OUTSIDE format: [xmin, ymin, xmax, ymax]
    if INSIDE[0] < OUTSIDE[0]:
        return False
    if INSIDE[1] < OUTSIDE[1]:
        return False
    if INSIDE[2] > OUTSIDE[2]:
        return False
    if INSIDE[3] > OUTSIDE[3]:
        return False
    return True

# returns 
def getShapefileData (INPUT_SHP, INPUT_BOUND_BOX, OFFSET):
    final_data = []
    driver = ogr.GetDriverByName("ESRI Shapefile")
    print(driver)
    print(INPUT_SHP)
    source = driver.Open(INPUT_SHP, 0)
    print(source)
    first_layer = source.GetLayer()
    
    counter = 0
    found = False
    found1 = False
    for i in range(0,len(first_layer)):
        feature = first_layer.GetFeature(i)
        attributes = json.loads(feature.ExportToJson())
        coords = attributes['geometry']['coordinates']
        xmin = coords[0][1][0]
        ymin = coords[0][0][1]
        xmax = coords[0][2][0]
        ymax = coords[0][1][1]
        box = [xmin, ymin, xmax, ymax]
        center_x, center_y = getCenter(box)
        xmin = center_x - OFFSET
        ymin = center_y - OFFSET
        xmax = center_x + OFFSET
        ymax = center_y + OFFSET
        box = [xmin, ymin, xmax, ymax]
        if checkBounds(box, INPUT_BOUND_BOX) == True:
            if found == False:
                print("Hit!")
                found = True
            dmglvl = attributes['properties']['damageleve']
            obj = {}
            obj['box'] = box
            obj['cls'] = dmglvl
            final_data.append(obj)
            counter += 1
            
        if i % 10000 == 0:
            print(i)
            if found1 == True:
                break
            if found == True:
                found1 = True
            
#         if i > 150000:
#             break
    print("Hits:", counter)
    return final_data

In [5]:
# IMAGES_DIR = dir of images
# INPUT_SHP = input shapefile
# OFFSET = half of final crop coordinate width/height
# DIM = pixel width/height of final crop
def process_images(IMAGES_DIR, INPUT_SHP, RESULT_DIR, OFFSET=0.00015, DIM=128):
    ALL_INFO = []
    if not os.path.exists(RESULT_DIR):
        subprocess.call(['mkdir', RESULT_DIR])
    with open(RESULT_DIR + '/' + "classes.txt", "a+") as classes:
        with open(RESULT_DIR + '/' + "images.txt", "a+") as images:
            for FILENAME in os.listdir(IMAGES_DIR):
                if FILENAME.endswith(".asm") or FILENAME.endswith(".py"): 
                     # print(os.path.join(directory, FILENAME))
                    continue
                elif FILENAME.endswith(".tif"):
                    INPUT_IMAGE = os.path.join(IMAGES_DIR, FILENAME)
                    INPUT_BOUND_BOX = getBoundingBox(INPUT_IMAGE)
                    print("Processing", FILENAME)
                    ALL_INFO = getShapefileData(INPUT_SHP, INPUT_BOUND_BOX, OFFSET)
                    for ITEM in ALL_INFO:
                        BOX = ITEM['box']
                        CLS = ITEM['cls']
                        xmin = BOX[0]
                        ymin = BOX[1]
                        xmax = BOX[2]
                        ymax = BOX[3]
                        x, y = getCenter(BOX)
                        final_file = 'img_' + str(x) + '_' + str(y) + '.tif'
                        subprocess.call(['gdalwarp', '-t_srs', 'EPSG:4326', '-te', \
                                         str(xmin), str(ymin), str(xmax), str(ymax), \
                                         '-ts', str(DIM), str(DIM), INPUT_IMAGE, RESULT_DIR + '/' + final_file])
                        classes.write(CLS + '\n')
                        images.write(final_file + '\n')
                else:
                    continue
    print("complete")

In [6]:
images_dir = 'batch2'
shapefile_name = '/Users/maxwu/ibm/DamageDetection/Satellite-Analysis/boundingboxes-all-damagearea-pixelcoords.shp'
RESULT_DIR = 'results_2'
OFFSET = 0.00015
DIM = 128

process_images(images_dir, shapefile_name, RESULT_DIR)

Processing 20170903aC0954200w293900n.tif
<osgeo.ogr.Driver; proxy of <Swig Object of type 'OGRDriverShadow *' at 0x10b6da630> >
/Users/maxwu/ibm/DamageDetection/Satellite-Analysis/boundingboxes-all-damagearea-pixelcoords.shp
<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x1089ad180> >
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
Hits: 0
Processing 20170903aC0954630w293430n.tif
<osgeo.ogr.Driver; proxy of <Swig Object of type 'OGRDriverShadow *' at 0x10b6da420> >
/Users/maxwu/ibm/DamageDetection/Satellite-Analysis/boundingboxes-all-damagearea-pixelcoords.shp
<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataS

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
Hits: 0
Processing 20170903aC0950600w293130n.tif
<osgeo.ogr.Driver; proxy of <Swig Object of type 'OGRDriverShadow *' at 0x10b6da7e0> >
/Users/maxwu/ibm/DamageDetection/Satellite-Analysis/boundingboxes-all-damagearea-pixelcoords.shp
<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x1089ad150> >
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
Hit!
150000
160000
Hits: 702
Processing 20170903aC0955230w294330n.tif
<osgeo.ogr.Driver; proxy of <Swig Object of type 'OGRDriverShadow *' at 0x10b6da8d0> >
/Users/maxwu/ibm/DamageDetection/Satellit

Hits: 0
Processing 20170903aC0953730w293000n.tif
<osgeo.ogr.Driver; proxy of <Swig Object of type 'OGRDriverShadow *' at 0x10b6da7e0> >
/Users/maxwu/ibm/DamageDetection/Satellite-Analysis/boundingboxes-all-damagearea-pixelcoords.shp
<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x1089ad150> >
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
Hit!
130000
140000
Hits: 67
Processing 20170903aC0953000w292700n.tif
<osgeo.ogr.Driver; proxy of <Swig Object of type 'OGRDriverShadow *' at 0x10b6da8d0> >
/Users/maxwu/ibm/DamageDetection/Satellite-Analysis/boundingboxes-all-damagearea-pixelcoords.shp
<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x1089ad0c0> >
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
37

In [7]:
# # image that will be cropped
# INPUT_IMAGE = 'ligma_og.tif'
# # shapefile of buildings
# INPUT_SHP = 'boundingboxes-all-damagearea-pixelcoords.shp'
# # coordinate width/height divided by 2
# OFFSET = 0.00015
# # pixel dimensions of output
# DIM = 128